# How-To, First Steps

## Load package and create basic options

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustfpm.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [2]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [4]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <robustfpm.finance.derivatives.AmericanOption object at 0x11463fdc0>
Option 2: <robustfpm.finance.derivatives.EuropeanOption object at 0x11463fd60>
Option 3: <robustfpm.finance.derivatives.BermudanOption object at 0x105a6b3d0>
Option 4: <robustfpm.finance.derivatives.AmericanOption object at 0x105a6b2e0>


In [5]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustfpm.pricing`

In [5]:
from robustfpm.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [6]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [7]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0806 sec (CPU 0.0806 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 64/90 (71.11%)
iter = 73/90 (81.11%)
iter = 84/90 (93.33%)
t = 3
iter = 4/67 (5.97%)
iter = 15/67 (22.39%)
iter = 16/67 (23.88%)
t = 2
iter = 5/45 (11.11%)
t = 1
t = 0
Computing value function in intermediate points in time: 0.4102 sec (CPU 0.4102 sec)
Solving the problem: 0.4912 sec (CPU 0.4912 sec)
Value: 5.327786420219619


Let's play around and change Lattice step

In [8]:
pm1.lattice = Lattice(delta=[.1])
sol2 = solver.solve(pm1)
print('Value: {0}'.format(sol2['Vf'][0][0]))

Precalculating points for value function evaluation: 0.5837 sec (CPU 0.5836 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 27/818 (3.30%)
iter = 248/818 (30.32%)
iter = 284/818 (34.72%)
iter = 322/818 (39.36%)
iter = 332/818 (40.59%)
iter = 360/818 (44.01%)
iter = 382/818 (46.70%)
iter = 416/818 (50.86%)
iter = 470/818 (57.46%)
iter = 474/818 (57.95%)
iter = 497/818 (60.76%)
iter = 523/818 (63.94%)
iter = 603/818 (73.72%)
iter = 604/818 (73.84%)
iter = 659/818 (80.56%)
iter = 693/818 (84.72%)
t = 3
iter = 200/609 (32.84%)
iter = 265/609 (43.51%)
iter = 295/609 (48.44%)
iter = 351/609 (57.64%)
iter = 379/609 (62.23%)
iter = 413/609 (67.82%)
t = 2
iter = 15/405 (3.70%)
iter = 78/405 (19.26%)
iter = 104/405 (25.68%)
iter = 144/405 (35.56%)
iter = 184/405 (45.43%)
iter = 248/405 (61.23%)
iter = 270/405 (66.67%)
iter = 280/405 (69.14%)
iter = 295/405 (72.84%)
iter = 332/405 (81.98%)
iter = 382/405 (94.32%)
iter = 403/405 (99.51%)
t = 1
iter = 75/20

Let's try 2D Problem with another option and *additive* dynamics.

In [9]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 200
sol3 = solver.solve(pm2)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Precalculating points for value function evaluation: 0.9343 sec (CPU 0.9343 sec)
Computing value function in the last point: 0.0005 sec (CPU 0.0004 sec)
t = 4
iter = 241/5913 (4.08%)
iter = 696/5913 (11.77%)
iter = 799/5913 (13.51%)
iter = 802/5913 (13.56%)
iter = 1030/5913 (17.42%)
iter = 1174/5913 (19.85%)
iter = 1211/5913 (20.48%)
iter = 1313/5913 (22.21%)
iter = 1463/5913 (24.74%)
iter = 1590/5913 (26.89%)
iter = 1819/5913 (30.76%)
iter = 1940/5913 (32.81%)
iter = 2070/5913 (35.01%)
iter = 3523/5913 (59.58%)
iter = 3835/5913 (64.86%)
iter = 3962/5913 (67.00%)
iter = 4232/5913 (71.57%)
iter = 4570/5913 (77.29%)
iter = 4696/5913 (79.42%)
iter = 4700/5913 (79.49%)
iter = 4726/5913 (79.93%)
iter = 4744/5913 (80.23%)
iter = 5135/5913 (86.84%)
iter = 5308/5913 (89.77%)
iter = 5388/5913 (91.12%)
iter = 5722/5913 (96.77%)
iter = 5897/5913 (99.73%)
t = 3
iter = 335/3355 (9.99%)
iter = 523/3355 (15.59%)
iter = 644/3355 (19.20%)
iter = 886/3355 (26.41%)
iter = 1381/3355 (41.16%)
iter = 1622/3